<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- Setup
- Binding machine learning engine
- Subscriptions
- Performance monitor, scoring and payload logging
- Quality monitor and feedback logging
- Fairness, Drift monitoring and explanations

<a id="setup"></a>
## 1. Setup

### Requirements installation

In [ ]:
!pip install sagemaker --no-cache | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade boto3 --no-cache | tail -n 1
!pip install --upgrade pandas --no-cache | tail -n 1

**Action:** Restart the kernel.

### Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

- Download this [notebook](https://github.com/IBM/watson-openscale-samples/blob/draft_v1/IBM%20Cloud/AWS%20Sagemaker/notebooks/Credit%20%20model%20with%20SageMaker%20linear-learner%20.ipynb) to create SageMaker model
- Run the notebook to train a SageMaker model and create deployment endpoint for online inference

### 1.2 Authentication

Import and initiate.

#### ACTION: Get OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```bash
ibmcloud target -g <myResourceGroup>
```
- get details of the instance
```bash
ibmcloud resource service-instance 'AI-OpenScale-instance_name'
```

In [ ]:
CLOUD_API_KEY = '***'

In [ ]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [ ]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

In [ ]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_API_KEY_ID = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [ ]:
BUCKET_NAME = "***" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training_recoded.csv"

In [ ]:
!rm credit_risk_training_recoded.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_training_recoded.csv"

### Store training data in COS for OpenScale reference

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [ ]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

### Initiate OpenScale client

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment if using IAM token
wos_client = APIClient(authenticator=authenticator)
wos_client.version

Create schema for data mart.

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL, # For DB2 use DatabaseType.DB2
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

In [ ]:
wos_client.data_marts.get(data_mart_id).result.to_dict()

<a id="binding"></a>
## 2. Bind machine learning engines

### Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [ ]:
SAGEMAKER_ENGINE_CREDENTIALS = {
                   'access_key_id': '***', 
                   'secret_access_key': '***', 
                   'region': '***'}


In [ ]:
SERVICE_PROVIDER_NAME = "AWS Machine Learning"
SERVICE_PROVIDER_DESCRIPTION = "Added by AWS tutorial WOS notebook."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
added_service_provider_result=wos_client.service_providers.add(
        name="AWS",
        description="AWS Service Provider",
        service_type=ServiceTypes.AMAZON_SAGEMAKER,
        credentials=SageMakerCredentials(
            access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
            secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'],
            region=SAGEMAKER_ENGINE_CREDENTIALS['region']
        ),
        background_mode=False
    ).result



service_provider_id = added_service_provider_result.metadata.id
print("Service Provider id ", service_provider_id)

In [ ]:
wos_client.service_providers.show()

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

In [ ]:
deployment_id='684e35eee8a479470cee05983e1f9d64' # get GUID for related deployment
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break
model_asset_details

<a id="subsciption"></a>
## 3. Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [ ]:
aws_asset = Asset(
        asset_id=model_asset_details['entity']['asset']['asset_id'],
        name=model_asset_details['entity']['asset']['name'],
        url=model_asset_details['entity']['asset']['url'],
        asset_type=model_asset_details['entity']['asset']['asset_type'] if 'asset_type' in model_asset_details['entity']['asset'] else 'model',
        problem_type=ProblemType.BINARY_CLASSIFICATION,
        input_data_type=InputDataType.STRUCTURED,
    )

In [ ]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url'] )

In [ ]:
deployment = AssetDeploymentRequest(
        deployment_id=model_asset_details['metadata']['guid'],
        url=model_asset_details['metadata']['url'],
        name=model_asset_details['entity']['name'],
        deployment_type=model_asset_details['entity']['type'],
        scoring_endpoint =  scoring_endpoint
    )

In [ ]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )

In [ ]:
feature_columns = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes', 'LoanDuration', 'LoanAmount', 'InstallmentPercent', 'CurrentResidenceDuration', 'Age', 'ExistingCreditsCount', 'Dependents']
categorical_columns = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes']

In [ ]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='predicted_label',
        probability_fields=['score'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=None,
        output_data_schema=None,
        feature_fields=feature_columns,
        categorical_fields=categorical_columns
    )

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=aws_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id

#### List subscriptions

In [ ]:
wos_client.subscriptions.show()

<a id="scoring"></a>
## 4. Performance metrics, scoring and payload logging

### Score the credit risk model and measure response time

In [ ]:
import requests
import time
import json
import boto3

In [ ]:
subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
subscription_details

In [ ]:
endpoint_name = subscription_details['entity']['deployment']['name']

payload = "0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,12,4152,2,3,29,2,1"

In [ ]:
runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

### Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with OpenScale standard.

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

In [ ]:
values = [float(s) for s in payload.split(',')]

request_data = {'fields': feature_columns, 
                'values': values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

In [ ]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [ ]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
feedback_dataset_id

The feedback records can be send to feedback table using below code.

In [ ]:
import requests

data = pd.read_csv('https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_feedback_recoded.csv',header=0,dtype=np.float)
feedback_columns = data.columns.tolist()
feedback_records = data.values.tolist()

payload_scoring =  [{"fields": feedback_columns, "values": feedback_records}]
wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)

In [ ]:
wos_client.data_sets.print_records_schema(data_set_id=feedback_dataset_id)

In [ ]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

<a id="datamart"></a>
## 6. Get the logged data

### Payload logging

#### Print schema of payload_logging table

In [ ]:
wos_client.data_sets.print_records_schema(data_set_id=payload_data_set_id)

<a id="fairness_and_explainability"></a>
## 7. Fairness, Drift monitoring and explanations

### Get payload data

In [ ]:
scoring_data_filename='credit_risk_scoring_recoded.csv'

In [ ]:
!rm credit_risk_scoring_recoded.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/draft_v1/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.csv"

In [ ]:
sm_runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

with open(scoring_data_filename) as f_payload:
    scoring_response = sm_runtime.invoke_endpoint(EndpointName = endpoint_name,
                                                  ContentType = 'text/csv',
                                                  Body = f_payload.read().encode())
    
    result = json.loads(scoring_response['Body'].read().decode())
    print(json.dumps(result, indent=2))

In [ ]:
request_data = {'fields': feature_columns, 
                'values': payload_values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}

In [ ]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

In [ ]:
wos_client.data_sets.show_records(payload_data_set_id)

### Enable and run fairness monitoring

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex_female",
         "majority": [[0,0]],
         "minority": [[1,1]],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": [0],
    "unfavourable_class": [1],
    "min_records": 30
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id

### Run fairness monitor

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

### Enable and run Drift monitoring

#### Drift requires a trained model to be uploaded manually for AWS. You can train, create and download a drift detection model using template given ( check for Drift detection model generation) [here](https://github.com/IBM-Watson/aios-data-distribution/blob/master/training_statistics_notebook.ipynb)

In [ ]:
!rm -rf creditrisk_aws_drift_detection_model.tar.gz
!wget -O creditrisk_aws_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/draft_v1/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz?raw=true 

In [ ]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_aws_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 30,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

### Enable Explainability and run explanation on sample record

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

Getting a `transaction_id` to run explanation on

In [ ]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

In [ ]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

In [ ]:
scoring_ids=['68ba9232-c526-4ae1-a05f-a9489258816d-18']
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

In [ ]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

## Congratulations

You have finished the tutorial for IBM Watson OpenScale and AWS Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit AWS model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

---